# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [19]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd

browser = webdriver.Firefox(executable_path='/usr/bin/geckodriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_visible_text('臺北市-士林')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()
# 取得資料（等瀏覽器資料出現後才執行）
time.sleep(2)
html_source = browser.page_source
soup = BeautifulSoup(html_source)
table = soup.find('table', class_='TABLE_G')
aq_dict = []
tr_list = table.find_all('tr')
for tr in tr_list:
    td_list = tr.find_all('td')
    if len(td_list) > 3:
        aq_dict.append({'監測項目': td_list[0].text, '單位': td_list[1].text, '監測日期': [], '監測值': []})
        aq_dict[-1]['監測日期'].append(td_list[2].text)
        aq_dict[-1]['監測值'].append(td_list[3].text)
    elif len(td_list) == 3:
        aq_dict[-1]['監測日期'].append(td_list[0].text)
        aq_dict[-1]['監測值'].append(td_list[1].text)
df = pd.DataFrame(aq_dict[0]['監測日期'][:8], columns={'監測日期'})
for item in aq_dict:
    if item['監測項目'] == 'SO2':
        df_ = pd.DataFrame(item['監測值'][:8], columns={'%s (%s)' % (item['監測項目'], item['單位'])})
        df = pd.concat([df, df_], axis=1, sort=False)
df

,監測日期,SO2 (ppb)
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [20]:
for item in aq_dict:
    if item['監測項目'] == 'CO':
        df_ = pd.DataFrame(item['監測值'][:8], columns={'%s (%s)' % (item['監測項目'], item['單位'])})
        df = pd.concat([df, df_], axis=1, sort=False)
df

,監測日期,SO2 (ppb),CO (ppm)
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30
